## PROJECT NAME: BANGALORE RESTAURANTS

### INTRODUCTION / BUSINESS PROBLEM

#### Bangalore is the IT Hub of Indian subcontinent where people from all over the country and from abroad come for education, business and securing good jobs. The commute services are good in the city but the major problem lies in the food cuisine and accomodation. As people from various ethnic groups come to Bangalore and search for a place to settle, they find out the there are not food services and restaurants that suits their cusine and taste. On the other hand, people who who try to setup there catering and food business in the city face a lot of competition if they are unaware of the food market. Following the problem points for various group of people:

#### a. Studies and professionals coming from other states or countries find it difficult to find a good place to eat their preferred cuisine.

#### b. Enterprenuers who wishes to start a new business do not understand the cuisine of city and where to open the restaurant, where certain cuisines are dominating, where they have greater opportunity to grow.

#### c. People who wants to try other cuisine find it difficult to narrow down the search for it and it beomes tedious to find right restaurant near them.

## Describe Data

### Data Preparation for the required business problem

#### a. The data for the problem needs the details of the restaurants like there name and location in the city of Bangalore.
#### b. The restaurants type and the main cuisine that they serve.

### In the following code cells Data preparation is going to be done using FourSquare APIs

In [1]:
# Installing required dependencies

!pip install bs4
!pip install requests
!pip install html5lib


In [2]:
# Importing requried libraries

import requests
import numpy as np
import pandas as pd


In [3]:
# Importing geopy for location

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')


Libraries imported.


In [4]:
# Import Folium for showing required city maps

!pip install folium
import folium # map rendering library

print('Folium installed')

Folium installed


In [5]:
# @hidden_cell
# Credentials for Foursquare API

CLIENT_ID = 'XXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXX' # your Foursquare Secret
VERSION = '20210705' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Credentails Available:')


Credentails Available:


In [6]:
# Getting Bangalore City coordinates to show the city map
# Run the cell multiple time for correct output

address = 'Bangalore, India'

geolocator = Nominatim(user_agent="Bangalore_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore City are 12.9791198, 77.5912997.


### Fetching data using FourSquare API

In [7]:
import json

url_explore = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v=VERSION,
ll=f'{latitude},{longitude}',
query='food',
limit=LIMIT,
radius=2000
)
resp_explore = requests.get(url=url_explore, params=params).json()["response"]['groups'][0]['items']
resp_explore


[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bc1cd90b492d13a4e74a660',
   'name': 'Toscano',
   'location': {'address': 'UB City Level 2 Concorde Block',
    'crossStreet': '24 Vittal Mallya Road',
    'lat': 12.97198038085137,
    'lng': 77.59606565238312,
    'labeledLatLngs': [{'label': 'display',
      'lat': 12.97198038085137,
      'lng': 77.59606565238312}],
    'distance': 948,
    'cc': 'IN',
    'city': 'Bangalore',
    'state': 'Karnātaka',
    'country': 'India',
    'formattedAddress': ['UB City Level 2 Concorde Block (24 Vittal Mallya Road)',
     'Bangalore',
     'Karnātaka',
     'India']},
   'categories': [{'id': '4bf58dd8d48988d110941735',
     'name': 'Italian Restaurant',
     'pluralName': 'Italian Restaurants',
     'shortName': 'Italian',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
      'suffix': '.png'}

In [14]:
# return only relevant information for each nearby venue

restaurant_list = []
restaurant_list.append([('Bangalore',
v['venue']['name'], 
v['venue']['location']['lat'], 
v['venue']['location']['lng'],  
v['venue']['categories'][0]['name'],
v['venue']['categories'][0]['pluralName'],
v['venue']['categories'][0]['shortName']) for v in resp_explore])

restaurant_list[0][0:3]

[('Bangalore',
  'Toscano',
  12.97198038085137,
  77.59606565238312,
  'Italian Restaurant',
  'Italian Restaurants',
  'Italian'),
 ('Bangalore',
  'Smoke House Deli',
  12.971656175823421,
  77.59825417986832,
  'Deli / Bodega',
  'Delis / Bodegas',
  'Deli / Bodega'),
 ('Bangalore',
  "Sunny's",
  12.972023446176495,
  77.59845399703514,
  'Italian Restaurant',
  'Italian Restaurants',
  'Italian')]

In [15]:
# Converting venues into dataframe

city_venues = pd.DataFrame([item for restaurant_list in restaurant_list for item in restaurant_list])
city_venues.columns = ['City',
              'RestaurantName', 
              'Latitude', 
              'Longitude', 
              'CategoryName', 
              'Type', 
              'Cuisine']

city_venues.sort_values('Cuisine', axis = 0, ascending = True, inplace = True)
city_venues.reset_index(drop = True, inplace = True)
city_venues.head()

,City,RestaurantName,Latitude,Longitude,CategoryName,Type,Cuisine
0,Bangalore,Samarkand,12.980616,77.604668,Afghan Restaurant,Afghan Restaurants,Afghan
1,Bangalore,Hard Rock Cafe Bengaluru,12.976179,77.601543,American Restaurant,American Restaurants,American
2,Bangalore,J W Kitchen,12.972410,77.594592,Asian Restaurant,Asian Restaurants,Asian
3,Bangalore,Bareerah,12.990762,77.595100,BBQ Joint,BBQ Joints,BBQ
4,Bangalore,Au Bon Pain,12.964394,77.589647,Bagel Shop,Bagel Shops,Bagels


### Incorporating additional details into data and refining it

In [16]:
print('There are {} uniques Cuisine.'.format(len(city_venues['Cuisine'].unique())))

# one hot encoding
city_venues_onehot = pd.get_dummies(city_venues[['Cuisine']], prefix="", prefix_sep="")

# Adding city columns at the end
city_venues_onehot['Cuisine'] = city_venues['Cuisine']

# Putting city column at 1st
fixed_columns = [city_venues_onehot.columns[-1]] + list(city_venues_onehot.columns[:-1])
city_venues_onehot = city_venues_onehot[fixed_columns]

city_venues_onehot.sort_values('Cuisine', axis = 0, ascending = True, inplace = True)
city_venues_onehot.reset_index(drop = True, inplace = True)
city_venues_onehot

There are 31 uniques Cuisine.


,Cuisine,Afghan,American,Asian,BBQ,Bagels,Bakery,Breakfast,Burgers,Café,...,Modern European,Parsi,Restaurant,Seafood,Snacks,South Indian,Steakhouse,Sushi,Vegetarian / Vegan,Vietnamese
0,Afghan,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,American,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Asian,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BBQ,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bagels,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Steakhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
96,Sushi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
97,Vegetarian / Vegan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
98,Vegetarian / Vegan,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
# Grouping rows by Cuisine and taking mean of frequency of occurence of each category

city_venues_grouped = city_venues_onehot.groupby('Cuisine').mean().reset_index()
city_venues_grouped.head()

,Cuisine,Afghan,American,Asian,BBQ,Bagels,Bakery,Breakfast,Burgers,Café,...,Modern European,Parsi,Restaurant,Seafood,Snacks,South Indian,Steakhouse,Sushi,Vegetarian / Vegan,Vietnamese
0,Afghan,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,American,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Asian,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BBQ,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bagels,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### The 1 stage of data preparation is completed.